# 装饰器
## 理解装饰器

理解装饰器，我们必须要理解当我们给一个函数加上装饰器的时候，真正发生了什么，比如下面的代码。

```python
@my_decorator
def my_func(*args, **kwargs):
    pass
```

实际上这一段代码在编译的时候，编译器真正执行的是
```python
def my_func(*args, **kwargs):
    pass
my_func = my_decorator(my_func)
```
简单来说，`my_func`函数先被定义，然后又被`my_decorator(my_func)`的返回值重新定义了。现在来看`my_decotator`:

- 首先它必须是一个可调用的对象，最常见的，必须是一个函数。它也可以是一个类，此为后话。
- 其次它必须且只能接受一个参数`my_fun`。
- 它必须有一个返回值，这个返回值用来覆盖原来的`my_fun`。按常理，返回值应该也是一个函数，只有这样，`my_fun`才能继续被当成函数调用。其实也不尽然，此为另一后话。


## 基本的装饰器

现在我们尝试自己写一个装饰器。


In [2]:
from __future__ import print_function
from __future__ import unicode_literals


def my_decorator(func): #接受一个函数参数
    def wrapper(*args, **kwargs):
        print("before {} ...".format(func.__name__))
        return_var = func(*args, **kwargs)
        print("After {} ...".format(func.__name__))
        return return_var
    return wrapper
@my_decorator
def my_func():
    """print myself"""
    print("In my_func")

my_func()      

before my_func ...
In my_func
After my_func ...


可以看到，在上述代码，我们在`my_decorator`里面定义了一个函数`wrapper`，`wrapper`接受任何参数，并将其传递给func并获得func的返回值。`my_decorator`的返回值为`wrapper`函数，用来覆盖`my_func`。此时打印`my_func`的名字，获得的是`wrapper`。

In [ ]:
print(my_func.__name__) #wrapper
print(my_func.__doc__) #None

正常情况下我们希望这些函数的性质，比如函数名，docstring等能够保留下来，此时需要`functools.wraps`：

In [ ]:
from functools import wraps

def my_decorator(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        print("before {} ...".format(func.__name__))
        return_var = func(*args, **kwargs)
        print("After {} ...".format(func.__name__))
        return return_var
    return wrapper
@my_decorator
def my_func():
    """print myself"""
    print("In my_func")

my_func()
print("function name is:", my_func.__name__)
print("function docstring is:", my_func.__doc__)

这里注意到`wrapers`的调用方法有点不一样，使用的是`@wraps(func)`的形式。这意味着`wraps`是一个**返回decorator的函数。**，这种情况下可以更加灵活的生成`decorator`，比如:

In [ ]:
def run_verbose(verbose=False):
    def my_decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            if verbose is True:
                print("going to run ", func.__name__)
            return_var = func(*args, **kwargs)
            if verbose is True:
                print("ended function ", func.__name__)
            return return_var
        return wrapper
    return my_decorator
    
@run_verbose(True)
def my_func1():
    print("running in my_func1")

@run_verbose()
def my_func2():
    print("running in my_func2")
             
my_func1()
print()
print()
my_func2()

以上这些基本涵盖了python装饰器90%的用法。以下是一些高级主题

## 基于类的装饰器

之前创建的装饰器都基于函数。实际上根据装饰器的用法，我们只需要保证其

- 支持`decorator(myfunc)`的调用形式
- 返回值也可以通过相似的方法调用

试想我们平时创建一个类的对象，使用的方法是`obj = myclass(args)`的形式，因此我们可以创建一个基于类的装饰器。此时要考虑的问题是如何使返回的`obj`支持`obj(*args, **kwargs)`的函数调用形式，因为一般情况下我们通过`obj.method()`的形式来使用这个返回的对象。解决方法也很简单，只需要给myclass定义`__call__`函数，那么就可以通过`obj()`的方式_运行_这个对象。现在我们来定义一个基于类的装饰器。


In [ ]:
class MyDecorator(object):
    def __init__(self, func):
        self.func = func
    
    def __call__(self, *args, **kwargs):
        print("going to run", self.func.__name__)
        return_var = self.func(*args, **kwargs)
        print("ended function", self.func.__name__)
    
@MyDecorator    
def my_func():
    """my func for decorator"""
    print("running my_func")
    
my_func()
print(my_func.__doc__)
print(my_func.__name__)

运行以上代码，除了输出我们意料的“going to run my_func”等之外， 最后的`__doc__`为`None`，`__name__`会报错，显然此时`my_func`是一个`MyDecorator`对象，因此没有原来函数的这些性质，此时可以使用`functools.update_wrapper(self, func)`来获得原函数的相关性质。

## 类装饰器

前面所有的装饰器都是用来装饰函数，实际上装饰器还可以用在类上，从而修改类的基本行为，比如如下类装饰器。 

In [3]:
def dump_class_vars(cls):
    for k, v in cls.__dict__.iteritems():
        print(k, v)
    return cls

@dump_class_vars
class Fo(object):
    def __init__(self):
        pass
    
    def dump_me(self):
        pass

('__module__', '__main__')
('dump_me', <function dump_me at 0x7f7a9c13ded8>)
('__dict__', <attribute '__dict__' of 'Fo' objects>)
('__weakref__', <attribute '__weakref__' of 'Fo' objects>)
('__doc__', None)
('__init__', <function __init__ at 0x7f7a9c13dc80>)


此时装饰器的写法和之前的有一些不一样，并没有返回任何函数，而是对传入的`class`做了一些操作，然后返回原`class`就结束了。  